In [2]:
import datatable as dt
import redis
import pandas as pd
import json
import random
from datetime import datetime
from redis.commands.json.path import Path
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redisearch import TextField, NumericField, TagField, Query

### Connect to Redis

In [3]:
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

### Load the data into a DataTable object

In [5]:
data = dt.fread("data/cleaned_data.csv")

### Convert DataTable to a list of dictionaries

In [95]:
df = data.to_pandas()
# df = df[["key","cvr_no", "company_name","industry_designation_primary"]]


df = df.drop(columns=['C0'])

df[:5]

,key,timestamp,advertisements,address_id,access_address_id,company_name,cvr_no,no_of_units,employees,employees_range,...,y,longitude,latitude,joint_status,date_of_incorporation,homepage,yearly_report_start,yearly_report_end,yearly_result,scrapped
0,2,2022-10-06 11:21:50,1.0,f9c3bc56-7e49-4ce1-b689-1893e35bed57,49953400-9077-4dd1-90d5-f6889aadf746,GRASS DK ApS,41201134,1,\N,\N,...,554204,9.89260622,57.01340725,NORMAL,2020-02-17,-,2021-01-01 00:00:00,2021-12-31 00:00:00,4255,\N
1,3,2022-10-06 11:21:50,1.0,0a3f50c9-d7fb-32b8-e044-0003ba298018,0a3f509c-274c-32b8-e044-0003ba298018,Bomholt Malerfirma ApS,40317430,1,1,ANTAL_1_1,...,556968,9.93806351,57.01048807,NORMAL,2019-03-05,www.bomholtmalerfirma.dk,2020-07-01 00:00:00,2021-06-30 00:00:00,18812,1
2,4,2022-10-06 11:21:50,1.0,0a3f50ca-b735-32b8-e044-0003ba298018,0a3f509c-b65e-32b8-e044-0003ba298018,"BO ANDERSEN HOLDING, SVENSTRUP ApS",33047754,1,\N,\N,...,549430,9.81295876,56.96558049,NORMAL,2010-06-28,-,2021-01-01 00:00:00,2021-12-31 00:00:00,703252,\N
3,5,2022-10-06 11:21:50,NaN,0a3f50ca-260d-32b8-e044-0003ba298018,0a3f509c-58d9-32b8-e044-0003ba298018,Fyenbo Holding ApS,41026162,1,\N,\N,...,555493,9.91473184,57.0494713,NORMAL,2019-12-12,-,2021-01-01 00:00:00,2021-12-31 00:00:00,-52881,\N
4,7,2022-10-06 11:21:50,NaN,0a3f50ca-33af-32b8-e044-0003ba298018,0a3f509c-6287-32b8-e044-0003ba298018,Kjeld Johannesen Holding ApS,37537535,1,\N,\N,...,542364,9.6980118,57.03392186,NORMAL,2016-03-08,-,2021-01-01 00:00:00,2021-12-31 00:00:00,1486249,\N


In [ ]:
# df.head()
df = df[["key","timestamp","cvr_no", "company_name","industry_designation_primary"]]
df.head()

In [96]:
# Create the 'key_transform' column using the apply method
df['key_transform'] = df.index.to_series().apply(lambda x: f'{x}')

In [97]:
df.head()

,key,timestamp,advertisements,address_id,access_address_id,company_name,cvr_no,no_of_units,employees,employees_range,...,longitude,latitude,joint_status,date_of_incorporation,homepage,yearly_report_start,yearly_report_end,yearly_result,scrapped,key_transform
0,2,2022-10-06 11:21:50,1.0,f9c3bc56-7e49-4ce1-b689-1893e35bed57,49953400-9077-4dd1-90d5-f6889aadf746,GRASS DK ApS,41201134,1,\N,\N,...,9.89260622,57.01340725,NORMAL,2020-02-17,-,2021-01-01 00:00:00,2021-12-31 00:00:00,4255,\N,0
1,3,2022-10-06 11:21:50,1.0,0a3f50c9-d7fb-32b8-e044-0003ba298018,0a3f509c-274c-32b8-e044-0003ba298018,Bomholt Malerfirma ApS,40317430,1,1,ANTAL_1_1,...,9.93806351,57.01048807,NORMAL,2019-03-05,www.bomholtmalerfirma.dk,2020-07-01 00:00:00,2021-06-30 00:00:00,18812,1,1
2,4,2022-10-06 11:21:50,1.0,0a3f50ca-b735-32b8-e044-0003ba298018,0a3f509c-b65e-32b8-e044-0003ba298018,"BO ANDERSEN HOLDING, SVENSTRUP ApS",33047754,1,\N,\N,...,9.81295876,56.96558049,NORMAL,2010-06-28,-,2021-01-01 00:00:00,2021-12-31 00:00:00,703252,\N,2
3,5,2022-10-06 11:21:50,NaN,0a3f50ca-260d-32b8-e044-0003ba298018,0a3f509c-58d9-32b8-e044-0003ba298018,Fyenbo Holding ApS,41026162,1,\N,\N,...,9.91473184,57.0494713,NORMAL,2019-12-12,-,2021-01-01 00:00:00,2021-12-31 00:00:00,-52881,\N,3
4,7,2022-10-06 11:21:50,NaN,0a3f50ca-33af-32b8-e044-0003ba298018,0a3f509c-6287-32b8-e044-0003ba298018,Kjeld Johannesen Holding ApS,37537535,1,\N,\N,...,9.6980118,57.03392186,NORMAL,2016-03-08,-,2021-01-01 00:00:00,2021-12-31 00:00:00,1486249,\N,4


In [30]:
df['key_transform']

0      0
1      1
2      2
3      3
4      4
      ..
95    95
96    96
97    97
98    98
99    99
Name: key_transform, Length: 100, dtype: object

In [40]:
trans_key = df['key'].astype('str')
df['key'] = trans_key
df.head()

,key,timestamp,advertisements,address_id,access_address_id,company_name,cvr_no,no_of_units,employees,employees_range,...,y,longitude,latitude,joint_status,date_of_incorporation,homepage,yearly_report_start,yearly_report_end,yearly_result,scrapped
0,2,2022-10-06 11:21:50,1.0,f9c3bc56-7e49-4ce1-b689-1893e35bed57,49953400-9077-4dd1-90d5-f6889aadf746,GRASS DK ApS,41201134,1,\N,\N,...,554204,9.89260622,57.01340725,NORMAL,2020-02-17,-,2021-01-01 00:00:00,2021-12-31 00:00:00,4255,\N
1,3,2022-10-06 11:21:50,1.0,0a3f50c9-d7fb-32b8-e044-0003ba298018,0a3f509c-274c-32b8-e044-0003ba298018,Bomholt Malerfirma ApS,40317430,1,1,ANTAL_1_1,...,556968,9.93806351,57.01048807,NORMAL,2019-03-05,www.bomholtmalerfirma.dk,2020-07-01 00:00:00,2021-06-30 00:00:00,18812,1
2,4,2022-10-06 11:21:50,1.0,0a3f50ca-b735-32b8-e044-0003ba298018,0a3f509c-b65e-32b8-e044-0003ba298018,"BO ANDERSEN HOLDING, SVENSTRUP ApS",33047754,1,\N,\N,...,549430,9.81295876,56.96558049,NORMAL,2010-06-28,-,2021-01-01 00:00:00,2021-12-31 00:00:00,703252,\N
3,5,2022-10-06 11:21:50,NaN,0a3f50ca-260d-32b8-e044-0003ba298018,0a3f509c-58d9-32b8-e044-0003ba298018,Fyenbo Holding ApS,41026162,1,\N,\N,...,555493,9.91473184,57.0494713,NORMAL,2019-12-12,-,2021-01-01 00:00:00,2021-12-31 00:00:00,-52881,\N
4,7,2022-10-06 11:21:50,NaN,0a3f50ca-33af-32b8-e044-0003ba298018,0a3f509c-6287-32b8-e044-0003ba298018,Kjeld Johannesen Holding ApS,37537535,1,\N,\N,...,542364,9.6980118,57.03392186,NORMAL,2016-03-08,-,2021-01-01 00:00:00,2021-12-31 00:00:00,1486249,\N


In [ ]:
df.info()

In [ ]:
df.industry_designation_primary.tolist()

In [98]:
df.key_transform = df.key_transform.astype(int)
df.dtypes

key                                      int32
timestamp                       datetime64[ns]
advertisements                         float64
address_id                              object
access_address_id                       object
company_name                            object
cvr_no                                   int32
no_of_units                              int32
employees                               object
employees_range                         object
employees_date                          object
road_code                              float64
road_name                               object
house_no                               float64
letter                                  object
floor                                   object
door                                    object
postal_code                            float64
postal_code_name                        object
municipality_name                       object
municipality_code                        int32
region       

### Upload the data to Redis

In [113]:
for _, row in df.iterrows():
    row_id = row['key_transform']  # Assuming 'key' column contains the unique identifier
    row_dict = row.drop('key_transform').to_dict()
    row_dict = {k: v if pd.notna(v) else None for k, v in row_dict.items()}
    # json_data = json.dumps(row_dict)
    print(row_dict)
    # Convert Timestamp values to string
    for key, value in row_dict.items():
        if isinstance(value, pd.Timestamp):
            row_dict[key] = value.strftime('%Y-%m-%d %H:%M:%S')
    
    r.execute_command('JSON.SET', row_id, '.', json.dumps(row_dict))
    # json_data = json.dumps(row_dict)

print("Data uploaded to Redis successfully.")

### Test the data retrieval from Redis

In [ ]:
result = r.json().get('2')
print(result)

In [89]:
# df.head()
df = df[["key","timestamp","cvr_no", "company_name","industry_designation_primary"]]
df.head()

,key,timestamp,cvr_no,company_name,industry_designation_primary
0,2,2022-10-06 11:21:50,41201134,GRASS DK ApS,Engroshandel med kemiske produkter
1,3,2022-10-06 11:21:50,40317430,Bomholt Malerfirma ApS,Malerforretninger
2,4,2022-10-06 11:21:50,33047754,"BO ANDERSEN HOLDING, SVENSTRUP ApS",Ikke-finansielle holdingselskaber
3,5,2022-10-06 11:21:50,41026162,Fyenbo Holding ApS,Ikke-finansielle holdingselskaber
4,7,2022-10-06 11:21:50,37537535,Kjeld Johannesen Holding ApS,Ikke-finansielle holdingselskaber


In [154]:
trans_key = df['key'].astype('str')
df['key'] = trans_key
df.head()

,key,cvr_no,company_name,industry_designation_primary
0,2,41201134,GRASS DK ApS,Engroshandel med kemiske produkter
1,3,40317430,Bomholt Malerfirma ApS,Malerforretninger
2,4,33047754,"BO ANDERSEN HOLDING, SVENSTRUP ApS",Ikke-finansielle holdingselskaber
3,5,41026162,Fyenbo Holding ApS,Ikke-finansielle holdingselskaber
4,7,37537535,Kjeld Johannesen Holding ApS,Ikke-finansielle holdingselskaber


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   key                           100 non-null    object
 1   cvr_no                        100 non-null    int32 
 2   company_name                  100 non-null    object
 3   industry_designation_primary  100 non-null    object
dtypes: int32(1), object(3)
memory usage: 2.9+ KB


### Define a schema

In [128]:
schema = (
    TextField("$.key", as_name="key"),
    NumericField("$.cvr_no", as_name="cvr_no"),
    TextField("$.company_name", as_name="company_name"),
    TextField("$.industry_designation_primary", as_name="industry_designation_primary")
)

### Set index name

In [129]:
index_name = "idx:cleaned_data"

### Create an index

In [130]:
rs = r.ft(index_name)
rs.create_index(
    schema,
    definition=IndexDefinition(
        prefix=["cleaned_data:"], index_type=IndexType.JSON
    )
)

'OK'

### Upload data to Redis and index it

In [131]:
for _, row in df.iterrows():
    row_id = row['key']
    row_dict = row.drop('key').to_dict()

    r.json().set(f"cleaned_data:{row_id}", Path.root_path(), row_dict)
    
print("Data stored and indexed successfully.")

Data stored and indexed successfully.


### Perform a search query

In [10]:
result = r.json().get('10005')
print(result)

{'key': 11665, 'timestamp': '2022-10-06 11:22:25', 'advertisements': 1.0, 'address_id': '0a3f50cb-0406-32b8-e044-0003ba298018', 'access_address_id': '0a3f509c-e642-32b8-e044-0003ba298018', 'company_name': 'JANCA ApS', 'cvr_no': 21173797, 'no_of_units': 1, 'employees': '\\N', 'employees_range': '\\N', 'employees_date': '\\N', 'road_code': 9432.0, 'road_name': 'Volsted Skolevej', 'house_no': 105.0, 'letter': '', 'floor': '', 'door': '', 'postal_code': 9530.0, 'postal_code_name': 'Støvring', 'municipality_name': 'AALBORG', 'municipality_code': 851, 'region': 'Region Nordjylland', 'telephone_no': '-', 'email': '-', 'form_of_business': 'Anpartsselskab', 'form_of_business_type': 'APS', 'company_form_code': 80, 'industry_code_primary': 642020.0, 'industry_designation_primary': 'Ikke-finansielle holdingselskaber', 'industry_group': 64, 'branch_code_1': '-', 'industry_designation_1': '-', 'branch_code_2': '-', 'industry_designation_2': '-', 'branch_code_3': '-', 'industry_designation_3': '-', '

### Generate dummy data

In [169]:
# Define the number of dummy rows to generate
num_rows = 100

# Create a list of random values for each column
keys = range(1, num_rows+1)
company_names = [f"Company {i}" for i in range(1, num_rows+1)]
cvr_nos = random.sample(range(10000000, 99999999), num_rows)
industry_designations = [f"Industry {random.randint(1, 36)}" for _ in range(num_rows)]

# Create a DataFrame from the generated data
df = pd.DataFrame({
    'key': keys,
    'company_name': company_names,
    'cvr_no': cvr_nos,
    'industry_designation_primary': industry_designations
})

# Display the generated dummy data
df

# Save the DataFrame as a CSV file
df.to_csv('data/my_data.csv')
